## FINE-TUNE Bert on a QA task (SQUAD 1.1)

## Install and import necessary libraries

In [ ]:
! pip install datasets transformers

     |████████████████████████████████| 312 kB 9.2 MB/s 
     |████████████████████████████████| 3.5 MB 13.6 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 133 kB 35.8 MB/s 
     |████████████████████████████████| 243 kB 29.8 MB/s 
     |████████████████████████████████| 895 kB 29.1 MB/s 
     |████████████████████████████████| 596 kB 36.8 MB/s 
     |████████████████████████████████| 6.8 MB 35.2 MB/s 
     |████████████████████████████████| 144 kB 20.9 MB/s 
     |████████████████████████████████| 271 kB 23.2 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers
from datasets import load_dataset, load_metric, ClassLabel, Sequence, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
import random
import pandas as pd
from IPython.display import display, HTML
import collections
from tqdm.auto import tqdm
import numpy as np
import torch

In [ ]:
# some initializations

squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Load datasets

In [ ]:
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
# train dataset
train = datasets["validation"]

train

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

In [ ]:
# validation dataset
validation = datasets["train"].select(range(1000))

validation

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1000
})

In [ ]:
def get_random_elements(dataset, num_examples=200):
    """
    A function to randomly get and visualize 200 examples from a
    dataset object to use them for inference purposes
    """
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    #display(HTML(df.to_html()))
    return Dataset.from_pandas(df)

In [ ]:
# test dataset
test = get_random_elements(datasets["train"])

test

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 200
})

## Preprocess data

In [ ]:
# instantiate a tokenizer to tokenize the inputs (including converting the tokens to their corresponding
# IDs in the pretrained vocabulary) and put it in a format the model expects,
# as well as generate the other inputs that model requires

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting is needed.
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        # truncate only the context, not the question
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        # our tokenizer can automatically return us a list of features capped by
        # a certain maximum length, with the overlap
        return_overflowing_tokens=True,
        #  we will also need to to map parts of the original context to some tokens
        return_offsets_mapping=True,
        padding="max_length")

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
# apply function to all sentences (or pairs of sentences) in our training dataset
# since preprocessing changes the number of samples, we need to remove the old columns when applying it
# batche=True: encode the texts by batches together

tokenized_train = train.map(prepare_train_features, batched=True, remove_columns=train.column_names)

  0%|          | 0/11 [00:00<?, ?ba/s]

In [ ]:
len(tokenized_train)

10784

In [ ]:
# apply function to all sentences (or pairs of sentences) in our validation dataset
# since preprocessing changes the number of samples, we need to remove the old columns when applying it
# batche=True: encode the texts by batches together

tokenized_val = validation.map(prepare_train_features, batched=True, remove_columns=validation.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
len(tokenized_val)

1032

## Fine-tune model

In [ ]:
# download the pretrained model
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [ ]:
# folder name which will be used to save the checkpoints of the model
model_name = model_checkpoint.split("/")[-1]

# define all the attributes to customize the training
args = TrainingArguments(
    f"/content/{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01)

In [ ]:
# data collator that will batch our processed examples together
data_collator = default_data_collator

In [ ]:
# instantiate a Trainer objects and pass all needed arguments
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# model training (finetuning)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10784
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2022


Epoch,Training Loss,Validation Loss
1,3.024300,1.590448
2,1.647100,1.473043
3,1.157200,1.503966


Saving model checkpoint to /content/distilbert-base-uncased-finetuned-squad/checkpoint-500
Configuration saved in /content/distilbert-base-uncased-finetuned-squad/checkpoint-500/config.json
Model weights saved in /content/distilbert-base-uncased-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/distilbert-base-uncased-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/distilbert-base-uncased-finetuned-squad/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 16
Saving model checkpoint to /content/distilbert-base-uncased-finetuned-squad/checkpoint-1000
Configuration saved in /content/distilbert-base-uncased-finetuned-squad/checkpoint-1000/config.json
Model weights saved in /content/distilbert-base-uncased-finetuned-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/distilbert-base-uncased-finetuned-squad/checkpoint-1000/tokeni

TrainOutput(global_step=2022, training_loss=1.7912212696320697, metrics={'train_runtime': 2361.1586, 'train_samples_per_second': 13.702, 'train_steps_per_second': 0.856, 'total_flos': 3170166819176448.0, 'train_loss': 1.7912212696320697, 'epoch': 3.0})

In [ ]:
# save trained model
trainer.save_model("test-squad-trained")

Saving model checkpoint to test-squad-trained
Configuration saved in test-squad-trained/config.json
Model weights saved in test-squad-trained/pytorch_model.bin
tokenizer config file saved in test-squad-trained/tokenizer_config.json
Special tokens file saved in test-squad-trained/special_tokens_map.json


## Evaluate (200 random examples) - Question 3

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
# apply function to all sentences (or pairs of sentences) in our test dataset

tokenized_test = test.map(prepare_validation_features,batched=True,remove_columns=test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#  predictions for all features
# the output of the model is a dict-like object that contains the loss
# (since we provided labels), the start and end logits.
raw_predictions = trainer.predict(tokenized_test)

The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 202
  Batch size = 16


In [ ]:
# The Trainer hides the columns that are not used by the model
#(here example_id and offset_mapping which we will need for our post-processing)
# so we set them back

tokenized_test.set_format(type=tokenized_test.format["type"], columns=list(tokenized_test.features.keys()))

In [ ]:
# build a map from example index to its corresponding features indices

examples = test
features = tokenized_test

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
# apply post-processing function to our raw predictions

final_predictions = postprocess_qa_predictions(test, tokenized_test, raw_predictions.predictions)

Post-processing 200 example predictions split into 202 features.


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
# load the metric from the datasets library

metric = load_metric("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in test]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 53.0, 'f1': 69.03276066816}

## Inference - Question 2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("test-squad-trained")
model = AutoModelForQuestionAnswering.from_pretrained("test-squad-trained")

Didn't find file test-squad-trained/added_tokens.json. We won't load it.
loading file test-squad-trained/vocab.txt
loading file test-squad-trained/tokenizer.json
loading file None
loading file test-squad-trained/special_tokens_map.json
loading file test-squad-trained/tokenizer_config.json
loading configuration file test-squad-trained/config.json
Model config DistilBertConfig {
  "_name_or_path": "test-squad-trained",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.16.0",
  "vocab_size": 30522
}

loading weights file test-squad-trained/pytorch_model.bin
All mod

In [ ]:
text = """
Pink Floyd were an English rock band formed in London in 1964.
Gaining an early following as one of the first British psychedelic groups,
they were distinguished for their extended compositions, sonic experimentation,
philosophical lyrics and elaborate live shows.
They became a leading band of the progressive rock genre,
cited by some as the greatest progressive rock band of all time.
Pink Floyd were founded in 1964 by Syd Barrett (guitar, lead vocals),
Nick Mason (drums), Roger Waters (bass guitar, vocals), Richard Wright (keyboards, vocals)
and Bob Klose (guitars); Klose quit in 1965.
Under Barrett's leadership, they released two charting singles
and the successful debut album The Piper at the Gates of Dawn (1967).
Guitarist and vocalist David Gilmour joined in December 1967;
Barrett left in April 1968 due to deteriorating mental health.
Waters became the primary lyricist and thematic leader,
devising the concepts behind the band's peak success with the albums
The Dark Side of the Moon (1973), Wish You Were Here (1975),
Animals (1977) and The Wall (1979). The musical film based on The Wall,
Pink Floyd – The Wall (1982), won two BAFTA Awards.
      """



questions = ["When did Pink Floyd come out?",
             "What year did the band come out?",
             "Can you name group's members?",
             "Who was Pink Floyd's lyricist?",
             "Which are the greatest Pink Floyd songs?",
             "Was David Gilmour member of the band"]

for question in questions:

  inputs = tokenizer.encode_plus(question, text,  add_special_tokens=True, return_tensors='pt')
  input_ids = inputs["input_ids"].tolist()[0]
  text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  start_positions = torch.tensor([1])
  end_positions = torch.tensor([3])
  outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)

  loss = outputs.loss
  start_scores = outputs.start_logits
  end_scores = outputs.end_logits

  answer_start = torch.argmax(start_scores)
  answer_end = torch.argmax (end_scores)+1


  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

  print(f"Question: {question}")
  print(f"Answer: {answer}\n")

Question: When did Pink Floyd come out?
Answer: 1964

Question: What year did the band come out?
Answer: 1964

Question: Can you name group's members?
Answer: syd barrett

Question: Who was Pink Floyd's lyricist?
Answer: syd barrett

Question: Which are the greatest Pink Floyd songs?
Answer: progressive rock band of all time

Question: Was David Gilmour member of the band
Answer: syd barrett

